In [13]:
import os
import re
import yaml


In [14]:
# Imagin there is a variable named COGNITE_API_KEY in the env variables

In [15]:
def parse_config(path=None, data=None, tag='!ENV'):

    # pattern for global vars: look for ${word}
    pattern = re.compile('.*?\${(\w+)}.*?')
    loader = yaml.SafeLoader

    # the tag will be used to mark where to start searching for the pattern
    # e.g. somekey: !ENV somestring${MYENVVAR}blah blah blah
    loader.add_implicit_resolver(tag, pattern, None)

    def constructor_env_variables(loader, node):
        value = loader.construct_scalar(node)
        match = pattern.findall(value)  # to find all env variables in line
        if match:
            full_value = value
            for g in match:
                full_value = full_value.replace(
                    f'${{{g}}}', os.environ.get(g, g)
                )
            return full_value
        return value

    loader.add_constructor(tag, constructor_env_variables)

    if path:
        with open(path) as conf_data:
            return yaml.load(conf_data, Loader=loader)
    elif data:
        return yaml.load(data, Loader=loader)
    else:
        raise ValueError('Either a path or data should be defined as input')

In [2]:
# conf = parse_config(path='./test.yaml')
# conf

In [1]:
# val = 'api_key: !ENV ${COGNITE_API_KEY}'
# conf2 = parse_config(data=val)
# conf2